In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")

In [3]:
import geopandas as gpd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from config.config import BASE_PATH, PATH_TO_PATH_CONFIG_FILE
from src.utils import load_paths_from_yaml, replace_base_path
from src.modeling.encodings import convert_aspect_to_cardinal_direction, convert_canopy_cover_to_classes, convert_ffmc_to_classes
from src.modeling.bayesian_models import (create_model_ffmc_adjustment_aspect, 
                                          create_model_ffmc_adjustment_foresttype,
                                          create_model_ffmc_adjustment_canopy_cover)
from src.modeling.utils import temporal_train_test_split

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:

paths = load_paths_from_yaml(PATH_TO_PATH_CONFIG_FILE)
paths = replace_base_path(paths, BASE_PATH)

In [30]:
# read training data
train_data = gpd.read_file(paths["training_data"])
train_data = train_data.loc[:, ["ffmc", "aspect", "foresttype", "canopy_cov", "fire", "date"]]


In [33]:
# data preprocessing
train_data.dropna(inplace=True)
train_data["foresttype"] = train_data.foresttype.astype("int")
train_data["aspect_categorized"] = train_data.aspect.apply(convert_aspect_to_cardinal_direction).astype("int")
train_data["canopy_cover_categorized"] = train_data.canopy_cov.apply(convert_canopy_cover_to_classes).astype("int")


In [34]:
coords = {"aspect_groups": [0, 1, 2, 3, 4, 5, 6, 7], 
          "foresttype_groups": [0, 1, 2, 3, 4, 5, 6], 
          "canopy_cover_groups": [0, 1, 2, 3, 4]}

In [35]:
# Split data temporally 
# Older samples (70%) will be used for training; newer samples (30%) will be used for evaluation
train_df, test_df = temporal_train_test_split(train_data, "date", 0.7)
relevant_columns = ["ffmc", "foresttype", "aspect_categorized", "canopy_cover_categorized", "date"]
X_train, y_train = train_df.loc[:, relevant_columns], train_df.loc[:, "fire"]
X_test, y_test = test_df.loc[:, relevant_columns], train_df.loc[:, "fire"]

scaler = StandardScaler()
X_train["ffmc"] = scaler.fit_transform(X_train[["ffmc"]])
X_test["ffmc"] = scaler.transform(X_test[["ffmc"]])


In [36]:
model_aspect = create_model_ffmc_adjustment_aspect(X_train, y_train, coords)
model_foresttype = create_model_ffmc_adjustment_foresttype(X_train, y_train, coords)
model_canopy_cover = create_model_ffmc_adjustment_canopy_cover(X_train, y_train, coords)

with model_aspect:
    idata_aspect=pm.sample()

with model_foresttype:
    idata_foresttype=pm.sample()

with model_canopy_cover:
    idata_canopy_cover=pm.sample()


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_b1, sigma_b1, intercept, beta_ffmc, error_beta]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 161 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_b1, sigma_b1, intercept, beta_ffmc, error_beta]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 129 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_b1, sigma_b1, intercept, beta_ffmc, error_beta]


In [ ]:
# save model
import cloudpickle

dict_to_save = {'model': model_aspect,
                'idata': idata_aspect
                }

with open(f'../../models/ffmc_adjustment/model_aspect.pkl' , 'wb') as buff:
    cloudpickle.dump(dict_to_save, buff)


dict_to_save = {'model': model_foresttype,
                'idata': idata_foresttype
                }

with open(f'../../models/ffmc_adjustment/model_foresttype.pkl' , 'wb') as buff:
    cloudpickle.dump(dict_to_save, buff)


dict_to_save = {'model': model_canopy_cover,
                'idata': idata_canopy_cover
                }

with open(f'../../models/ffmc_adjustment/model_canopy_cover.pkl' , 'wb') as buff:
    cloudpickle.dump(dict_to_save, buff)

In [ ]:
# Interpretation of contrast between group level coefficients and population level coefficient of ffmc
# The population level beta specifies the impact of ffmc on forest fire ignition (theoretically this should be positive; with increasing ffmc the danger of forest fire ignition increases)
# 

In [19]:
az.summary(idata_canopy_cover, var_names=["beta_ffmc"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_ffmc[0],9.141,22.387,1.078,26.761,1.791,1.401,286.0,261.0,1.01
beta_ffmc[1],2.639,0.778,1.494,4.166,0.038,0.027,414.0,700.0,1.00
beta_ffmc[2],2.098,0.398,1.420,2.921,0.016,0.011,593.0,1073.0,1.01
beta_ffmc[3],2.067,0.263,1.620,2.594,0.012,0.008,496.0,706.0,1.00
beta_ffmc[4],1.578,0.122,1.347,1.815,0.005,0.003,680.0,587.0,1.01


In [18]:
print((idata_aspect.posterior.beta_ffmc - idata_aspect.posterior.mu_b1).mean(axis=(0, 1)).values)

[ 0.17605097 -0.29629205 -0.77074934 -0.03129043  0.59940299  0.10729073
  0.07869177  0.19337925]
[ 0.60103592 -0.00354114  0.22389765 -0.35657617 -0.1727104  -0.27366792
  0.66473309]
[ 7.15664568  0.65411939  0.11372197  0.08277428 -0.40691867]


In [21]:
# 0 = coniferous non pine
# 1 = coniferous with mixed pine
# 2 = pine pure
# 3 = coniferous deciduous mixed with pine
# 4 = coniferous_deciduous_mixed_non_pine
# 5 = deciduous pure
# 6 = low and no vegetation

print((idata_foresttype.posterior.beta_ffmc - idata_foresttype.posterior.mu_b1).mean(axis=(0, 1)).values)

[ 0.60103592 -0.00354114  0.22389765 -0.35657617 -0.1727104  -0.27366792
  0.66473309]


In [22]:
# 4 >80%
# 3 61-80%
# 2 41-60%
# 1 21-40%
# 0 ≤20%
print((idata_canopy_cover.posterior.beta_ffmc - idata_canopy_cover.posterior.mu_b1).mean(axis=(0, 1)).values)

[ 7.15664568  0.65411939  0.11372197  0.08277428 -0.40691867]
